<a href="https://colab.research.google.com/github/rafatb/Berlin_Airbnb_Price_Prediction/blob/main/Berlin_Airbnb_Price_Prediction_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install missingno
!pip install autoviz
!pip install geopy
!pip install scipy
!pandas_profiling
!pip install statsmodels
!pip install pyodbc
!pip install seaborn
!pip install nltk


In [6]:
import numpy as np
import os
import pandas as pd
#from pandas_profiling import ProfileReport
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import ensemble, tree, linear_model
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from scipy.stats import pearsonr
import pyodbc
import seaborn as sns
sns.set()
plt.style.use('ggplot')
import re
from scipy import stats
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import ks_2samp
from scipy.stats import norm
import missingno as msno
import warnings
import datetime
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
import nltk

In [50]:
df = pd.read_pickle('/content/drive/MyDrive/kaggle/berline/ready_for_eda.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23536 entries, 0 to 23535
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Listing ID                      23536 non-null  int64         
 1   Overall Rating                  18914 non-null  float64       
 2   Accuracy Rating                 18888 non-null  float64       
 3   Cleanliness Rating              18892 non-null  float64       
 4   Checkin Rating                  18870 non-null  float64       
 5   Communication Rating            18886 non-null  float64       
 6   Location Rating                 18871 non-null  float64       
 7   Value Rating                    18868 non-null  float64       
 8   Price                           23536 non-null  float64       
 9   Square Feet                     425 non-null    float64       
 10  Reviews                         23536 non-null  int64         
 11  re

In [10]:
df.shape

(23536, 35)

#Data Protocol
###Exporting results to the protocol

In [40]:
# Type of valaues
df.dtypes.to_excel("berline_datatype.xlsx",
             sheet_name='data_type')
# Maximum valaues
df.select_dtypes(include=['float','int','category']).max().to_excel("max_berline.xlsx",
             sheet_name='max')
# Minimum Values
df.select_dtypes(include=['float','int','category']).min().to_excel("min_berline.xlsx",
             sheet_name='min')
# Missing Values
df.isnull().sum(axis=0).to_excel("NA_berline.xlsx",
             sheet_name='NA')
# exporting results to the protocol
df.nunique().to_excel("unique_berline.xlsx",
             sheet_name='unique')

#Transform data

In [51]:
df['year'] = df['review_date'].dt.year
df['month'] = df['review_date'].dt.month
df['day'] = df['review_date'].dt.day
df.drop(columns=['review_date'], inplace=True)

### move out non relevant data for the

In [52]:
#get dummies
df_dummy = df[[col for col in df if np.isin(df[col].dropna().unique(), [0, 1]).all()]]
df_dummy.shape

(23536, 3)

In [53]:
#get all numeric data
cols = df.columns
df_num = df._get_numeric_data().columns.dropna().unique()
df_num

Index(['Listing ID', 'Overall Rating', 'Accuracy Rating', 'Cleanliness Rating',
       'Checkin Rating', 'Communication Rating', 'Location Rating',
       'Value Rating', 'Price', 'Square Feet', 'Reviews', 'Bedrooms', 'Beds',
       'Bathrooms', 'Host Response Rate', 'Is Superhost', 'Latitude',
       'Longitude', 'Accomodates', 'Guests Included', 'Min Nights',
       'Instant Bookable', 'Is Exact Location', 'Overall Rating Cleansed',
       'year', 'month', 'day'],
      dtype='object')

In [54]:
pd.set_option('display.max_columns', None)
df_num_non_dummy = df[list(set(df_num)-set(df_dummy.columns))]
df_num_non_dummy.drop(['Listing ID'], axis=1, inplace=True)
# df_num_non_dummy.shape

In [55]:
#get categorial data
df_categorial=df[list(set(cols) - set(df_num))]
df_categorial.head(100)

,Host Response Rate Cleansed,Property Type Cleansed,Room Type,Neighbourhood Grouped Cleansed,Host Response Time,neighbourhood,Neighborhood Group,Postal Code,Host Since,Property Type
0,Low Response,Apartment,Private room,Pankow,within a day,Prenzlauer Berg,Pankow,10437,09-16-08,Apartment
1,Low Response,Apartment,Entire home/apt,Pankow,within a day,Prenzlauer Berg,Pankow,10405,10-19-08,Apartment
2,Very High Response,Apartment,Private room,Pankow,within an hour,Prenzlauer Berg,Pankow,10437,05-16-09,Apartment
3,Very High Response,Apartment,Entire home/apt,Pankow,within a day,Prenzlauer Berg,Pankow,10405,08-25-09,Apartment
4,Very High Response,Apartment,Entire home/apt,Pankow,within a day,Prenzlauer Berg,Pankow,10437,11-18-09,Apartment
...,...,...,...,...,...,...,...,...,...,...
95,NaN,Apartment,Entire home/apt,Pankow,None,Prenzlauer Berg,Pankow,10409,06-26-11,Apartment
96,Very High Response,Apartment,Entire home/apt,Mitte,within a day,Mitte,Mitte,10179,06-27-11,Apartment
97,Moderate Response,Apartment,Entire home/apt,Friedrichshain-Kreuzberg,within a day,Kreuzberg,Friedrichshain-Kreuzberg,10999,12-06-10,Apartment
98,Very High Response,Apartment,Private room,Mitte,within a few hours,Moabit,Mitte,10551,06-06-11,Apartment


In [48]:
df_categorial['Is Superhostxxx'] = df_categorial['Is Superhost'].replace({'t': True, 'f': False})
df_categorial['Is Superhostxxx']

,Is Superhostxxx
0,False
1,False
2,True
3,False
4,False
...,...
23531,False
23532,False
23533,False
23534,False


In [ ]:
df_aggregated = df_aggregated[df_aggregated["Price"] < 600]

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline
AV = AutoViz_Class()

#AV = AutoViz()

# Specify the dataset and target column
# df_av = AV.AutoViz(
#     filename="",  # Leave empty if passing a DataFrame
#     df=df_aggregated,  # Your Pandas DataFrame
#     depVar="Price",  # Specify the target column
#     verbose=2
# )

AV.AutoViz(filename="", sep=",", depVar="Price", dfte=df_aggregated, max_cols_analyzed=10)


In [ ]:
# from geopy.distance import great_circle
# def distance_to_mid(lat, lon):
#     berlin_centre = (52.5027778, 13.404166666666667)
#     accommodation = (lat, lon)
#     return great_circle(berlin_centre, accommodation).km

# df_aggregated['Distance From Center Cleansed'] = df_aggregated.apply(lambda row: round(distance_to_mid(row['Latitude'], row['Longitude']), 1), axis=1)
# bins = [0, 0.5, 1, 2, 4, 8, 16]
# labels = ["Center", "Cercile1 ", "Cercile2", "Cercile4", "Cercile8", "Cercile16"]

# # Apply pd.cut() to create a new binned column
# df_aggregated["Distance From Center Grouped Cleansed"] = pd.cut(df_aggregated["Distance From Center Cleansed"], bins=bins, labels=labels, include_lowest=True)
# # df_aggregated.drop(columns=["Distance From Center Cleansed"], inplace=True)
# # df_aggregated.drop(columns=["Latitude","Longitude"], inplace=True)
